In [1]:
from utils.consts import *

from utils.utilities import (
    test, show_dep, get_data, get_nominal_subjects, get_prep, get_objects, get_root, _is_verb, get_aux,
    check_if_wh, get_csubj, get_amod, get_actual_subtree_for_finding_agg, get_subtree_list)

import pickle
import json
import random
import json
from IPython.display import Markdown, display
from pprint import pprint
import numpy as np

In [2]:
import spacy

# nlp_pipe = spacy.load('en_core_web_lg')
PIPE = spacy.load('en_coref_lg')

In [17]:
data_files = ['dev', 'test', 'train']

data_path = "./data/%s_dataset.json"
x_file = './op_data1/%s_op_X.pkl'
y_file = './op_data1/%s_op_Y.pkl'
idx_file = './op_data1/%s_op_idx.pkl'

In [22]:
class vdata:
    def __init__(self):
        self.agg_X = list()
        self.agg_Y = list()
        self.agg_IDX = list()

    def fill_data(self, act_adjs, act_op, idx):
        temp_x = np.array([])
        temp_y = np.array([])
        if act_adjs is not None and len(act_adjs) > 0 and act_op in cond_ops:
    #         print("idx : %d"%idx)
            if len(act_adjs) > 4:
                act_adjs = act_adjs[:4]
            for tok in act_adjs:
                if tok.has_vector:
                    temp_x = np.append(temp_x, tok.vector)
            temp_y = (cond_funcs == act_op).astype(np.float)
        else:
            temp_x = np.zeros((X_size))
            temp_y = (cond_funcs == "OP").astype(np.float)
        if temp_x.size < X_size:
            temp_x = np.append(temp_x, np.zeros((X_size - temp_x.size)))
    #     print("fill data : ")
    #     print(temp_y)

        self.agg_X.append(temp_x)
        self.agg_Y.append(temp_y)
        self.agg_IDX.append(idx)

    def dump_data(self, dt):
        v_x = np.array(self.agg_X)
        v_y = np.array(self.agg_Y)
        v_idx = np.array(self.agg_IDX)

        with open(x_file%dt, 'wb') as fx:
            pickle.dump(v_x, fx)

        with open(y_file%dt, 'wb') as fy:
            pickle.dump(v_y, fy)

        with open(idx_file%dt, 'wb') as fidx:
            pickle.dump(v_idx, fidx)

In [5]:
X_size = 1200
agg_X = list()
agg_Y = list()
agg_IDX = list()
cond_funcs = np.array(cond_ops)

def refresh_globals():
    agg_X = list()
    agg_Y = list()
    agg_IDX = list()

def fill_data(act_adjs, act_op, idx):
    temp_x = np.array([])
    temp_y = np.array([])
    if act_adjs is not None and len(act_adjs) > 0 and act_op in cond_ops:
#         print("idx : %d"%idx)
        if len(act_adjs) > 4:
            act_adjs = act_adjs[:4]
        for tok in act_adjs:
            if tok.has_vector:
                temp_x = np.append(temp_x, tok.vector)
        temp_y = (cond_funcs == act_op).astype(np.float)
    else:
        temp_x = np.zeros((X_size))
        temp_y = (cond_funcs == "OP").astype(np.float)
    if temp_x.size < X_size:
        temp_x = np.append(temp_x, np.zeros((X_size - temp_x.size)))
#     print("fill data : ")
#     print(temp_y)
    
    agg_X.append(temp_x)
    agg_Y.append(temp_y)
    agg_IDX.append(idx)

In [6]:
def get_verb_and_subj_pair(data_sample):
#     print("\n========================================================\n")
#     pprint(data_sample)
    # get the doc representation of the string
    doc = PIPE(data_sample['question'])
    # Find out the root of the given statement
    root = get_root(doc)
#     import pdb; pdb.set_trace()
    is_verb = _is_verb(root)
    aux = None
#     if not is_verb:
        # Some times the root can be a noun, in that case it'll always be associated to
        # an aux verb, and we gotta find it
    aux = get_aux(root)
    # If there is any aux found, we use it as our verb root
    if aux is not None and len(aux) > 0:
        root_verb = aux[0]
    else:
        # Else, we use our root as it is
        root_verb = root
    # Check if there is any csubj attached this means we have two wh nouns attached to
    # two different verbs one is asking and other qualifying.
    csubj_verb = get_csubj(root_verb)
    if csubj_verb is not None:
        root_verb = csubj_verb
#     print("ROOT : {}".format(root.text))
#     print("AUX : ")
#     pprint(aux)
    # nominals in this case is going to be a list of all the cols
    # in SELECT clause. If we find any nsubj attached to the root_verb we catch it here.
    nominals = get_nominal_subjects(root_verb)
    is_wh = False
    # If what we have found is not actually a noun but a question noun (wh-word)
    # TODO: In this case we may want to go on hunt for the actual nominal col.
    if len(nominals) != 0:
        is_wh = check_if_wh(nominals)
    if is_wh:
        # for now
        pass
#     print("SUBJ : ")
#     pprint(nominals)
    # Even after all this if there is no way the prog have found a nominal
    # find out if there is any prop attached to the verb.
    # If there is then we may look for prep_obj attached to it for the nominal.
    prep = None
    if len(nominals) == 0:
        prep = get_prep(root_verb)
#     print("PREP : ")
#     pprint(prep)
    pobj = None
    if prep is not None:
        pobj = get_objects(prep)
        nominals.append(pobj)
#     print("POBJ : ")
#     pprint(pobj)
#     show_dep(doc)
    return nominals

In [7]:
def get_verbs_other_than_root(doc):
    verbs = list()
    o_verbs = list()
    root = get_root(doc)
    for tok in doc:
        if tok.pos_ in VERB and tok.idx != root.idx:
            o_verbs.append(tok)
        elif tok.pos_ in VERB:
            verbs.append(tok)
#         print(tok.text, tok.pos_)
    return verbs, o_verbs

In [8]:
ADJECTIVES = ["amod", "acomp"]

def get_amod_n_acomp_for_verbs(vb):
    st = get_subtree_list(vb)
    mods = list()
    for each_tok in st:
        if each_tok.dep_ in ADJECTIVES:
            mods.append(each_tok)
            
    return mods

def get_op_mods(vb_list):
    op_mods = list()
    for each_vb in vb_list:
        op_mods.extend(get_amod_n_acomp_for_verbs(each_vb))
    return op_mods

def get_all_op_mods(r_verbs, o_verbs):
    all_op_mods = list()
    o_op_mods = get_op_mods(o_verbs)
    all_op_mods.extend(o_op_mods[::-1])
    if len(all_op_mods) == 0:
        r_op_mods = get_op_mods(r_verbs)
        all_op_mods.extend(r_op_mods[::-1])
    return all_op_mods

In [9]:
def get_actual_params(conds, r_verbs, o_verbs, all_op_mods):
    if len(conds) == 0:
        OP = 'OP'
    else:
        OP = conds[0][1]
        
    all_op_mods.extend(r_verbs)
    all_op_mods.extend(o_verbs)
#     pprint(all_op_mods)
#     print(OP)
    if len(all_op_mods) > 4:
        all_op_mods[:4]
    return all_op_mods, OP

In [10]:
def dump_data(dt):
    v_x = np.array(agg_X)
    v_y = np.array(agg_Y)
    v_idx = np.array(agg_IDX)

    with open(x_file%dt, 'wb') as fx:
        pickle.dump(v_x, fx)

    with open(y_file%dt, 'wb') as fy:
        pickle.dump(v_y, fy)
    
    with open(idx_file%dt, 'wb') as fidx:
        pickle.dump(v_idx, fidx)

In [11]:
def process_data(dt, vd):
    refresh_globals()
    print("Data type : ", dt)
    data = get_data(data_path%dt)
    print("DATA : ", len(data))
    count = 0
    for idx in range(len(data)):
    # idx = -1
    # while count  30:
    #     idx += 1
        conds = data[idx]['sql']['conds']
        try:
    #         if conds[0][1] == "=":
    #             continue
            subjs = get_verb_and_subj_pair(data[idx])
#             print("\n========================================================\n")
    #         pprint(data[idx]['question'])
    #         show_dep(PIPE(data[idx]['question']))
    #         print("SUBJs : ")
    #         pprint(subjs)
    #         print("SELECT : ")
    #         pprint(data[idx]['sql']['sel'])
    #         print("CONDS : ", conds)
            verbs, o_verbs = get_verbs_other_than_root(PIPE(data[idx]['question']))
#             print("VERBS : ")
#             pprint(verbs)
#             print("OTHER VERBS : ")
#             pprint(o_verbs)
#             print("test1")
            all_op_mods = get_all_op_mods(verbs, o_verbs)
#             print(all_op_mods)
#             print("test2")
            act_adjs, cond_op = get_actual_params(conds, verbs, o_verbs, all_op_mods)
#             print("test3")
#             pprint(act_adjs)
#             print(cond_op)
            vd.fill_data(act_adjs, cond_op, idx)
            count += 1
        except Exception:
    #         amod = get_amod(PIPE(data[idx]['question']))
    #         fill_data(amod, agg_func, idx)
            pass

    print("COUNT : ", count)
    #         pprint(data[idx])

In [23]:
vd = vdata()

In [24]:
vd.agg_X

[]

In [ ]:
for dt in data_files:
    vd = vdata()
    process_data(dt, vd)
    vd.dump_data(dt)

Data type :  dev
DATA :  8421
COUNT :  8288
Data type :  test
DATA :  15878
COUNT :  15635
Data type :  train
DATA :  56355


In [ ]:
prediction = sess.run(tf.argmax(output_layer,1), feed_dict={X: [img]})
print ("Prediction for test image:", np.squeeze(prediction))